In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [54]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.normalization import BatchNormalization

In [12]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [13]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [14]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\Admin\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [16]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [18]:
# Training the model
history_32_rms = model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 60s 8ms/step - loss: 1.3233 - acc: 0.4332 - val_loss: 1.1553 - val_acc: 0.4649
Epoch 2/30
7352/7352 [==============================] - 55s 7ms/step - loss: 1.0250 - acc: 0.5536 - val_loss: 1.0156 - val_acc: 0.5806
Epoch 3/30
7352/7352 [==============================] - 54s 7ms/step - loss: 0.8296 - acc: 0.6432 - val_loss: 0.7981 - val_acc: 0.6043
Epoch 4/30
7352/7352 [==============================] - 56s 8ms/step - loss: 0.7209 - acc: 0.6542 - val_loss: 0.7564 - val_acc: 0.5955
Epoch 5/30
7352/7352 [==============================] - 52s 7ms/step - loss: 0.6490 - acc: 0.6702 - val_loss: 0.7268 - val_acc: 0.6237
Epoch 6/30
7352/7352 [==============================] - 52s 7ms/step - loss: 0.6264 - acc: 0.6780 - val_loss: 0.7316 - val_acc: 0.6206
Epoch 7/30
7352/7352 [==============================] - 52s 7ms/step - loss: 0.5941 - acc: 0.7

In [19]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         3        0                   0   
SITTING                  0      416        72        0                   0   
STANDING                 0      109       423        0                   0   
WALKING                  0        0         0      448                  25   
WALKING_DOWNSTAIRS       0        0         0        8                 392   
WALKING_UPSTAIRS         0        1         1       15                  16   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            24  
SITTING                            3  
STANDING                           0  
WALKING                           23  
WALKING_DOWNSTAIRS                20  
WALKING_UPSTAIRS                 438  


In [20]:
score_32_rms = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 565us/step


In [21]:
score_32_rms

[0.5226698747499483, 0.8914149983033594]

### LSTM with 48 units

In [22]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(48, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 48)                11136     
_________________________________________________________________
dropout_2 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 294       
Total params: 11,430
Trainable params: 11,430
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])

In [24]:
# Training the model
history_48_adam = model.fit(X_train,
Y_train,
batch_size=batch_size,
validation_data=(X_test, Y_test),
epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 57s 8ms/step - loss: 1.4184 - acc: 0.3617 - val_loss: 1.3882 - val_acc: 0.3773
Epoch 2/30
7352/7352 [==============================] - 55s 7ms/step - loss: 1.2866 - acc: 0.4248 - val_loss: 1.1935 - val_acc: 0.5185
Epoch 3/30
7352/7352 [==============================] - 55s 7ms/step - loss: 1.1797 - acc: 0.4710 - val_loss: 1.1463 - val_acc: 0.4649
Epoch 4/30
7352/7352 [==============================] - 55s 8ms/step - loss: 0.9862 - acc: 0.5147 - val_loss: 1.0529 - val_acc: 0.4863
Epoch 5/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.8807 - acc: 0.5569 - val_loss: 1.6132 - val_acc: 0.4242
Epoch 6/30
7352/7352 [==============================] - 56s 8ms/step - loss: 0.9183 - acc: 0.5520 - val_loss: 1.0211 - val_acc: 0.4866
Epoch 7/30
7352/7352 [==============================] - 55s 7ms/step - loss: 0.8614 - acc: 0.5928 - val_loss: 1.1428 - val_acc: 0.4876
Epoch 8

In [25]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      395        85        8                   0   
STANDING                 0      100       426        5                   0   
WALKING                  0        0         0      460                   8   
WALKING_DOWNSTAIRS       0        0         0        1                  31   
WALKING_UPSTAIRS         0        0         0       24                  15   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            3  
STANDING                           1  
WALKING                           28  
WALKING_DOWNSTAIRS               388  
WALKING_UPSTAIRS                 432  


In [26]:
score_48_adam = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 637us/step


In [27]:
score_48_adam

[0.43165912415404384, 0.7648456057007126]

### LSTM with 64 units

In [28]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
optimizer='rmsprop',
metrics=['accuracy'])

In [30]:
# Training the model
history_64_rms = model.fit(X_train,
Y_train,
batch_size=batch_size,
validation_data=(X_test, Y_test),
epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 59s 8ms/step - loss: 1.2178 - acc: 0.4691 - val_loss: 1.0465 - val_acc: 0.5205
Epoch 2/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.8636 - acc: 0.6321 - val_loss: 0.7916 - val_acc: 0.6362
Epoch 3/30
7352/7352 [==============================] - 57s 8ms/step - loss: 0.6813 - acc: 0.7040 - val_loss: 0.6797 - val_acc: 0.7662
Epoch 4/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.5363 - acc: 0.8069 - val_loss: 0.6028 - val_acc: 0.8113
Epoch 5/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.4041 - acc: 0.8682 - val_loss: 0.4323 - val_acc: 0.8867
Epoch 6/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.3008 - acc: 0.9019 - val_loss: 0.5835 - val_acc: 0.8388
Epoch 7/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.2636 - acc: 0.9159 - val_loss: 0.5152 - val_acc: 0.8517
Epoch 8

In [31]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  3      382       103        1                   0   
STANDING                 0       82       447        3                   0   
WALKING                  0        0         0      492                   4   
WALKING_DOWNSTAIRS       0        0         0       35                 384   
WALKING_UPSTAIRS         0        0         0       75                  12   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 384  


In [32]:
score_64_rms = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 908us/step


In [33]:
score_64_rms

[0.8537755837855637, 0.8819138106549033]

### LSTM using multilayer with 64 units

In [34]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.7))
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.7))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 390       
Total params: 52,358
Trainable params: 52,358
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss='categorical_crossentropy',
optimizer='rmsprop',
metrics=['accuracy'])

In [36]:
# Training the model
history_64_64_rms = model.fit(X_train,
Y_train,
batch_size=batch_size,
validation_data=(X_test, Y_test),
epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 138s 19ms/step - loss: 1.1621 - acc: 0.4933 - val_loss: 0.9635 - val_acc: 0.5426
Epoch 2/30
7352/7352 [==============================] - 134s 18ms/step - loss: 0.8951 - acc: 0.5786 - val_loss: 0.7849 - val_acc: 0.6166
Epoch 3/30
7352/7352 [==============================] - 137s 19ms/step - loss: 0.7275 - acc: 0.6386 - val_loss: 0.8098 - val_acc: 0.6149
Epoch 4/30
7352/7352 [==============================] - 141s 19ms/step - loss: 0.7318 - acc: 0.6376 - val_loss: 0.8035 - val_acc: 0.6166
Epoch 5/30
7352/7352 [==============================] - 157s 21ms/step - loss: 0.7077 - acc: 0.6545 - val_loss: 0.8568 - val_acc: 0.6183
Epoch 6/30
7352/7352 [==============================] - 157s 21ms/step - loss: 0.6574 - acc: 0.6787 - val_loss: 1.2631 - val_acc: 0.5497
Epoch 7/30
7352/7352 [==============================] - 135s 18ms/step - loss: 0.6084 - acc: 0.7247 - val_loss: 0.6831 - val_acc: 

In [37]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 521        0        16        0                   0   
SITTING                  0      342       145        0                   0   
STANDING                 0       36       496        0                   0   
WALKING                  0        0         0      479                  10   
WALKING_DOWNSTAIRS       0        0         0        3                 417   
WALKING_UPSTAIRS         0        0         0       12                  17   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            4  
STANDING                           0  
WALKING                            7  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 442  


In [38]:
score_64_64_rms = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [39]:
score_64_64_rms

[0.3797549853132213, 0.9151679674244995]

### LSTM with 128 Units

In [55]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(128, input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
# Adding a dropout layer
model.add(Dropout(0.25))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 128)               70656     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 774       
Total params: 71,942
Trainable params: 71,686
Non-trainable params: 256
_________________________________________________________________


In [56]:
model.compile(loss='categorical_crossentropy',
optimizer='rmsprop',
metrics=['accuracy'])

In [61]:
# Training the model
history_128_rms = model.fit(X_train,
Y_train,
batch_size=batch_size,
validation_data=(X_test, Y_test),
epochs=35)

Train on 7352 samples, validate on 2947 samples
Epoch 1/35
7352/7352 [==============================] - 86s 12ms/step - loss: 0.1320 - acc: 0.9520 - val_loss: 0.3552 - val_acc: 0.9321
Epoch 2/35
7352/7352 [==============================] - 85s 12ms/step - loss: 0.1435 - acc: 0.9391 - val_loss: 0.4673 - val_acc: 0.9148
Epoch 3/35
7352/7352 [==============================] - 83s 11ms/step - loss: 0.1348 - acc: 0.9493 - val_loss: 0.3677 - val_acc: 0.9253
Epoch 4/35
7352/7352 [==============================] - 85s 12ms/step - loss: 0.1321 - acc: 0.9498 - val_loss: 0.4059 - val_acc: 0.9091
Epoch 5/35
7352/7352 [==============================] - 83s 11ms/step - loss: 0.1292 - acc: 0.9501 - val_loss: 0.5025 - val_acc: 0.9125
Epoch 6/35
7352/7352 [==============================] - 83s 11ms/step - loss: 0.1181 - acc: 0.9514 - val_loss: 0.4127 - val_acc: 0.9257
Epoch 7/35
7352/7352 [==============================] - 83s 11ms/step - loss: 0.1255 - acc: 0.9491 - val_loss: 0.6469 - val_acc: 0.8938


In [62]:
score_128_rms = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 4s 1ms/step


In [63]:
score_128_rms

[nan, 0.168306752629793]

In [48]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim)))
# model.add(BatchNormalization())
# Adding a dropout layer
model.add(Dropout(0.7))

model.add(LSTM(64))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 390       
Total params: 120,454
Trainable params: 120,454
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.compile(loss='categorical_crossentropy',
optimizer='rmsprop',
metrics=['accuracy'])

In [50]:
# Training the model
history_128_64_rms = model.fit(X_train,
Y_train,
batch_size=batch_size,
validation_data=(X_test, Y_test),
epochs=25)

Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 194s 26ms/step - loss: 1.1063 - acc: 0.5137 - val_loss: 0.8138 - val_acc: 0.6447
Epoch 2/25
7352/7352 [==============================] - 171s 23ms/step - loss: 0.7901 - acc: 0.6243 - val_loss: 1.3584 - val_acc: 0.4608
Epoch 3/25
7352/7352 [==============================] - 168s 23ms/step - loss: 0.7428 - acc: 0.6313 - val_loss: 0.7845 - val_acc: 0.6040
Epoch 4/25
7352/7352 [==============================] - 168s 23ms/step - loss: 0.7310 - acc: 0.6421 - val_loss: 0.9128 - val_acc: 0.5993
Epoch 5/25
7352/7352 [==============================] - 170s 23ms/step - loss: 0.8087 - acc: 0.6329 - val_loss: 1.2705 - val_acc: 0.4788
Epoch 6/25
7352/7352 [==============================] - 166s 23ms/step - loss: 0.9216 - acc: 0.6055 - val_loss: 0.7259 - val_acc: 0.6332
Epoch 7/25
7352/7352 [==============================] - 170s 23ms/step - loss: 0.5462 - acc: 0.7735 - val_loss: 0.4805 - val_acc: 

In [51]:
score_128_64_rms = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 7s 2ms/step


In [52]:
score_128_64_rms

[0.39368825139877356, 0.9029521547336274]

In [60]:
# Creating table using PrettyTable library
from prettytable import PrettyTable

# Names of models
names =['1 LSTM layer with 32 Units(Optimizer-->rmsprop)','1 LSTM layer with 48 Units(Optimizer--> adam',\
'1 LSTM layer with 64 Units(Optimizer-->rmsprop)', '2 LSTM layer with 64 Units(Optimizer-->rmsprop)', \
'1 LSTM layer with 128 Units(Optimizer-->rmsprop)', '2 LSTM layer with 128 & 64 Units(Optimizer-->rmsprop)']
        
# Training accuracies
train_acc = [history_32_rms.history['acc'][29], history_48_adam.history['acc'][29], history_64_rms.history['acc'][29], 
             history_64_64_rms.history['acc'][29], history_128_rms.history['acc'][24], history_128_64_rms.history['acc'][24]]
             
# Test accuracies
test_acc =[score_32_rms[1], score_48_adam[1], score_64_rms[1], score_64_64_rms[1], score_128_rms[1], score_128_64_rms[1]]
numbering = [1,2,3,4, 5, 6]
           
# Initializing prettytable
ptable = PrettyTable()
# Adding columns
ptable.add_column("S.NO.",numbering)
ptable.add_column("MODEL",names)
ptable.add_column("Training Accuracy",train_acc)
ptable.add_column("Test Accuracy",test_acc)
# Printing the Table
print(ptable)

+-------+-------------------------------------------------------+--------------------+--------------------+
| S.NO. |                         MODEL                         | Training Accuracy  |   Test Accuracy    |
+-------+-------------------------------------------------------+--------------------+--------------------+
|   1   |    1 LSTM layer with 32 Units(Optimizer-->rmsprop)    | 0.9435527747551686 | 0.8914149983033594 |
|   2   |      1 LSTM layer with 48 Units(Optimizer--> adam     | 0.8333786724700761 | 0.7648456057007126 |
|   3   |    1 LSTM layer with 64 Units(Optimizer-->rmsprop)    | 0.9454570184983678 | 0.8819138106549033 |
|   4   |    2 LSTM layer with 64 Units(Optimizer-->rmsprop)    | 0.9547062023939065 | 0.9151679674244995 |
|   5   |    1 LSTM layer with 128 Units(Optimizer-->rmsprop)   | 0.9504896626768227 | 0.9284017645062775 |
|   6   | 2 LSTM layer with 128 & 64 Units(Optimizer-->rmsprop) | 0.9502176278563657 | 0.9029521547336274 |
+-------+-------------------

### Conclusion

* Tried multiple architecures and got best 92.84% accuracy using 1 layer architecture of LSTM with 128 Units
* Accuracy is better using RMSProp optimizer compared to Adam